In [1]:
import numpy as np
from scipy import stats
import pandas as pd

In [2]:
def load_result(file_name):
    data = [np.loadtxt("results/results_"+file_name+".txt", delimiter=",").mean(axis=0),	
            np.loadtxt("results/results_"+file_name+".txt", delimiter=",").std(axis=0)]
    return data

base_data = load_result("base")
baseSoft_data = load_result("baseSoft")
paste_data = load_result("paste")
pasteSoft_data = load_result("pasteSoft")
pix_data = load_result("pix")
pixSoft_data = load_result("pixSoft")

In [5]:
# paired t-test for each with each for accuracy
k=3 # 0 - accyracy, 1 - precision, 2 - recall, 3 - balanced accuracy 

model = [base_data, baseSoft_data, paste_data, pasteSoft_data, pix_data, pixSoft_data]
model_names = ['base', 'baseSoft', 'paste', 'pasteSoft', 'pix', 'pixSoft']
t_stat = pd.DataFrame(index=model_names, columns=model_names, dtype=float)
p_values = pd.DataFrame(index=model_names, columns=model_names, dtype=float)
for i in range(len(model_names)):
    for j in range(len(model_names)):
        t, p = stats.ttest_ind_from_stats(mean1=model[i][0][k], std1=model[i][1][k], nobs1=5, 
                                    mean2=model[j][0][k], std2=model[j][1][k], nobs2=5)
        t_stat.loc[model_names[i], model_names[j]] = t
        t_stat.loc[model_names[j], model_names[i]] = -t
        p_values.loc[model_names[i], model_names[j]] = p
        p_values.loc[model_names[j], model_names[i]] = p
t_stat_binary = t_stat.applymap(lambda x: 1 if x > 0 else 0)
p_values_binary = p_values.applymap(lambda x: 1 if x < 0.05 else 0)
final_results = t_stat_binary * p_values_binary

print("t>0, p<0.05")
final_results

t>0, p<0.05


,base,baseSoft,paste,pasteSoft,pix,pixSoft
base,0,0,0,0,0,0
baseSoft,1,0,1,0,0,0
paste,1,0,0,0,0,0
pasteSoft,1,1,1,0,1,0
pix,1,0,0,0,0,0
pixSoft,1,1,1,0,1,0


Student's t-test for balanced accuracy  
At the 0.05 confidence level, the PixSoft method is statistically significantly better than all methods except PasteSoft. PasteSoft is statistically significantly better than all methods except PixSoft. BaseSoft is better than Paste. All methods are better than Base.